In [ ]:
import os
def runningPods(cluster, app_name):
    output = os.popen('sudo kubectl top pod --context=' + cluster).read()
    lines = output.split("\n")
    numPods = 0
    which_app = len(app_name)
    for line in lines[:-1]:
        items = line.split()
        if len(items[0]) > 8 and items[0][:which_app] == app_name:
            numPods += 1
    return numPods
runningPods("cluster1-cntx","firewall"+'s')

In [ ]:
import os
def getchar(string, n):
    return str(string)[n - 1]
def cpuPods(cluster, app_name):
    output = os.popen('kubectl top pod --context=' + cluster).read()
    lines = output.split("\n")
    which_app = len(app_name)
    cpu = ''
    cpu_u=0
    limits=0
    for line in lines[:-1]:
        items = line.split()
        if items[0][:which_app] == app_name:
            if getchar(items[0], which_app) == 's':
                limits = 600
                cpu = items[1]
                cpu = cpu[:-1]
                cpu = int(cpu)
                cpu_u=cpu_u + cpu
            if getchar(items[0], which_app) == 'm':
                limits = 5*600
                cpu = items[1]
                cpu = cpu[:-1]
                cpu = int(cpu)
                cpu_u=cpu_u + cpu
            if getchar(items[0], which_app) == 'l':
                limits = 10*600
                cpu = items[1]
                cpu = cpu[:-1]
                cpu = int(cpu)
                cpu_u=cpu_u + cpu

        
                
#    print(limits - cpu_u)
    return limits - cpu_u


cpuPods("cluster1-cntx","firewall"+'m')


In [ ]:
#Which service is running 
def getchar(string, n):
    return str(string)[n - 1]
def whichService(cluster, app_name):
    size = []
    output = os.popen('sudo kubectl get services --context=' + cluster).read()
    lines = output.split("\n")
    which_app = len(app_name)
    for line in lines[:-1]:
        items = line.split()
        if items[0][:which_app] == app_name:
            size.append(getchar(items[0], which_app+1))
    #        print(size)
    return size
whichService("cluster1-cntx","firewall")
    

In [ ]:
def orderService(cluster, app_name):
    delay = []
    size = whichService(cluster, app_name)
    for s in range(len(size)):
        delay.append(bf[app_name][app_name+size[s]])
    sorted_delay = [x for _,x in sorted(zip(delay,size))]
    return sorted_delay
 #   print(delay)
                
    
orderService("cluster1-cntx","firewall")
    

In [ ]:
import os
def getIP(cluster, app_name):
    output = os.popen('kubectl get service -o wide --context=' + cluster).read()
    lines = output.split("\n")
    serviceIP = ''
    port=''
    which_app = len(app_name)
    for line in lines[:-1]:
        items = line.split()
        if items[0][:which_app] == app_name:
            serviceIP=items[2]+":"+items[4][:4]
    return serviceIP
getIP("cluster1-cntx","firewall"+'m')

In [ ]:
G = {"cluster1-cntx":["cluster1-cntx", "cluster2-cntx"], "cluster2-cntx":["cluster2-cntx", "cluster1-cntx"]}#, "cluster3-cntx":["cluster3-cntx"]}
link_latency = {"cluster1-cntx":{"cluster1-cntx":1, "cluster2-cntx":10},"cluster2-cntx":{"cluster2-cntx":1, "cluster1-cntx":10}}
                                 
bf = {"firewall":{"firewalls":10, "firewallm":5, "firewalll":1}, "encrypt":{"encrypts":10, "encryptm":5, "encryptl":1}, "decrypt":{"decrypts":10, "decryptm":5, "decryptl":1}}
G_req = {"firewall":["encrypt"], "encrypt":["firewall"]}#, "decrypt":["firewall"]}
usecase = {1:{"firewall":300,"encrypt": 300, "decrypt":300, "SLA":10}, 2:{"firewall":100,"encrypt": 100}, "SLA": 100}
u =2
#G = {"cluster1-cntx":["cluster1-cntx", "cluster2-cntx"], "cluster2-cntx":["cluster2-cntx", "cluster1-cntx"]}
#link_latency = {"cluster1-cntx":{"cluster1-cntx":1, "cluster2-cntx":100}, "cluster2-cntx":{"cluster2-cntx":1, "cluster1-cntx":100}, "cluster3-cntx":{"cluster2-cntx":10, "cluster1-cntx":10}}


In [ ]:
def orderlink(cluster):
    delay = []
    cnx = []
    for key in range(len(list(link_latency["cluster1-cntx"].keys()))):
        delay.append(link_latency["cluster1-cntx"][list(link_latency["cluster1-cntx"].keys())[key]])
        cnx.append(list(link_latency["cluster1-cntx"].keys())[key])
    sorted_delay = [x for _,x in sorted(zip(delay,cnx))]
    return sorted_delay
 #   print(delay)
                
    
orderlink("cluster1-cntx")

In [ ]:
def clusterCPU(cluster):
    cpu=0
    cpu_cl=0
    output = os.popen('sudo kubectl top nodes --context=' + cluster).read()
    lines = output.split("\n")
    for line in lines[:-1]:
        items = line.split()
        if items[0] != "NAME":
            cpu = items[1]
            cpu = cpu[:-1]
            cpu = int(cpu)
            cpu_cl = cpu_cl + cpu
    #        print(size)
    return 2000 - cpu_cl

clusterCPU('cluster1-cntx')

In [ ]:
def clusterMem(cluster):
    mem=0
    mem_cl=0
    nodes=0
    output = os.popen('sudo kubectl top nodes --context=' + cluster).read()
    lines = output.split("\n")
    for line in lines[:-1]:
        items = line.split()
        if items[0] != "NAME":
            mem = items[3]
            mem = mem[:len(mem)-2]
            nodes=nodes+1
            mem = int(mem)
            mem_cl = mem_cl + mem
    print(nodes)
    print(mem_cl)
    return nodes*1915 - mem_cl

clusterMem('cluster2-cntx')

In [ ]:
def place(i, q, s):
#    i = 'cluster2-cntx'
    if s == 's':
        os.system('sudo kubectl apply -f ~/SOCKS-deployement/'+q+'-S.yaml --context=' + i)
    elif s == 'm':
        os.system('sudo kubectl apply -f ~/SOCKS-deployement/'+q+'-M.yaml --context=' + i)
    elif s == 'l':
        os.system('sudo kubectl apply -f ~/SOCKS-deployement/'+q+'-L.yaml --context=' + i)
    

In [ ]:
def deployHeuristic(i, q):
#    req = 1
#    cl = 1
    proxy_conf='' 
    size=[]
    avail=0
    cpu_q= usecase[u][q]
    r = runningPods(i, q)
    if r >= 1: #there's a function already running on this cluster 
        size = orderService(i,q) #which function size is running? order by delay
        for s in range(len(size)): 
            avail = cpuPods(i, q+size[s]) #the available CPU depends on the service configuration
            if avail > cpu_q: #if the available resources > profiled resources requested 
                proxy_conf=getIP(i,q+size[s]) #then get IP to assign the service to proxy
                break
    elif r ==0 : #no service running 
        proxy_conf=''          
    #print(proxy_conf)
    return proxy_conf

def newService(i,q):
    proxy_conf=''
    cpu_q= usecase[u][q]
    r = runningPods(i, q)
    if 600 > cpu_q and runningPods(i, q+'s') == 0 and 600 < clusterCPU(i): #deploy new small service
        place(i,q,'s')
        proxy_conf=getIP(i,q+'s')       
    elif 5*600 > cpu_q and runningPods(i, q+'m')==0 and 5*600 < clusterCPU(i): #deploy new medium service
        proxy_conf=getIP(i,q+'m')
        proxy_conf=getIP(i,q+'m')
    elif 10*600 > cpu_q and runningPods(i, q+'l')==0 and 10*600 < clusterCPU(i): #deploy new large service
        proxy_conf=getIP(i,q+'l')
        proxy_conf=getIP(i,q+'l')
    elif proxy_conf=='':
        print("placement failed")
        
    
    #print(proxy_conf)
    return proxy_conf

In [ ]:
def checkdeploy(app_name):
    cl = 0
    p=0
    cluster=[]
    while p == 0:
        for cl in range(len(G)):
            i = list(G.keys())[cl]
            output = os.popen('sudo kubectl get services --context=' +i).read()
            lines = output.split("\n")
            which_app = len(app_name)
            for line in lines[:-1]:
                items = line.split()
                if items[0][:which_app] == app_name:
                    cluster.append(i)
                    p=1
                    
    return cluster
checkdeploy("encrypt")
            

In [ ]:
def proxyConfig(IP):
    newChain = []
    for ip in range(len(IP)):
        newChain.append('"-c",'+'"socks6://'+IP[ip]+'",')
    newChain = ''.join(newChain)
    newChain = newChain[:-1]
    print(newChain)
    os.system("sudo kubectl patch deployment proxy --namespace default --type='json' -p='[{'op': 'replace', 'path': '/spec/template/spec/containers/0/args', 'value': ["+newChain+"]}]'")

In [ ]:
IPs = []
m_cl= ''
m_ser=[]
OG = orderlink('cluster1-cntx') #try the fastest link first
for cl in range(len(OG)): #go over clusters 
    i = list(G.keys())[cl]
    for req in range(len(G_req)):
        q = list(G_req.keys())[req] #go over functions to deploy
        if req == 0 and q not in m_ser: #The first function can be deployed on any cluster
            if deployHeuristic(i,q) != "": #there is atleast one good candidate service running on i 
                IPs.append(deployHeuristic(i,q))
                m_cl= i #to check connection with next deployment
                m_ser.append(q)
                continue                
        elif m_cl in G[i] and q not in m_ser: #connection with previous bf and not already deployed
            if deployHeuristic(i,q) != "": #can be deployed
                IPs.append(deployHeuristic(i,q))
                m_cl= i
                m_ser.append(q)
                continue
    if len(m_ser) < len(G_req): #not all deployed yet we try a different cluster      
        continue 
    else:
        break
if len(m_ser) < len(G_req): #not all functions could be deployed 
    for cl in range(len(OG)): #go over clusters 
        i = list(G.keys())[cl]
        for req in range(len(G_req)):
            q = list(G_req.keys())[req] #go over functions to deploy
            if req == 0 and q not in m_ser:
                if newService(i,q) != '':
                    IPs.append(newService(i,q))
                    m_cl= i
                    m_ser.append(q)
                    continue
            elif m_cl in G[i] and q not in m_ser: #connection with previous bf and not already deployed
                    if newService(i,q) != '':
                        IPs.append(newService(i,q))
                        m_cl= i
                        m_ser.append(q)
                        continue
    
proxyConfig(IPs)
print(IPs)
print(m_ser)

In [1]:
def checkSLA():
    #compare expected latency vs real latency vs SLA
    
    

SyntaxError: unexpected EOF while parsing (<ipython-input-1-b4a87e32a509>, line 4)